----
## Compilado de exercicios do Curso de Amazon Bedrock
    Este compilado possui apenas uma copia dos scripts que foram executados na plataforma Lambda da AWS e nos arquivos .py, eles não foram executados neste notebook.
---------

### Seção 5: Criação de imagens utilizando Bedrock, API Gateway e S3.

In [ ]:
import json
#1. import boto3
import boto3
import base64
import datetime

#2. Criar conexão do cliente com os serviços Bedrock e S3 – Link  
client_bedrock = boto3.client('bedrock-runtime')
client_s3 = boto3.client('s3')

def lambda_handler(event, context):
#3. Armazenar os dados de entrada (prompt) em uma variável  
    input_prompt=event['prompt']
    print(input_prompt)

#4. Criar uma sintaxe de requisição para acessar o serviço Bedrock  
    response_bedrock = client_bedrock.invoke_model(contentType='application/json', accept='application/json',modelId='amazon.titan-image-generator-v2:0',
    body =json.dumps( {
        "taskType": "TEXT_IMAGE",
        "textToImageParams": {"text": input_prompt,      
        "negativeText": "none"
    },
            "imageGenerationConfig": {
            "quality": "standard",
            "numberOfImages": 1,
            "height": 512,
            "width": 512,
            "cfgScale": 10.0,
            "seed":42 
                }
    }))
    
    print(response_bedrock)

#5. 5a. Recuperar do dicionário, 5b. Converter o corpo do streaming para bytes usando json load, 5c. Imprimir  
    response_bedrock_byte=json.loads(response_bedrock['body'].read())
    print(response_bedrock_byte)
    
#6. 6a. Recuperar dados com a chave do artefato, 6b. Importar Base64, 6c. Decodificar de Base64  
    response_bedrock_base64 = response_bedrock_byte['images'][0]
    response_bedrock_finalimage = base64.b64decode(response_bedrock_base64)
    print(response_bedrock_finalimage)
    
#7. 7a. Fazer upload do arquivo para o S3 usando o método Put Object – Link  
    poster_name = 'posterName'+ datetime.datetime.today().strftime('%Y-%M-%D-%M-%S')
        response_s3=client_s3.put_object(
        Bucket='posterfilme01',
        Body=response_bedrock_finalimage,
        Key=poster_name)

#8. Gerar URL pré-assinada  
    generate_presigned_url = client_s3.generate_presigned_url('get_object', Params={'Bucket':'posterfilme01','Key':poster_name}, ExpiresIn=3600)
    print(generate_presigned_url)


    return {
        'statusCode': 200,
        'body': generate_presigned_url #json.dumps('Hello from Lambda!')
    }


### Seção 6: Criação de Resumos utilizando Bedrock, API Gateway e Lambda.

In [ ]:
import json
import boto3

# Criar cliente para o Bedrock
client_bedrock = boto3.client('bedrock-runtime')

def lambda_handler(event, context):
    # Obter o input do evento
    input_prompt = event['prompt']
    # print(input_prompt)

    # Criar a requisição para o Bedrock
    client_bedrockrequest = client_bedrock.invoke_model(
          modelId= "anthropic.claude-3-haiku-20240307-v1:0",
          contentType= "application/json",
          accept= "application/json",
          body= json.dumps({
            "anthropic_version": "bedrock-2023-05-31",
            "max_tokens": 400,
            "messages": [
              {
                "role": "user",
                "content": [
                  {
                    "type": "text",
                    "text": input_prompt
                  }
                ]
              }
            ]
          })
        )
     
    # Ler e processar a resposta do Bedrock
    client_bedrock_byte = client_bedrockrequest['body'].read()
    client_bedrock_string = json.loads(client_bedrock_byte)

    # Extrair a resposta do modelo
    client_final_response2 = client_bedrock_string['content'][0]['text']
    # client_final_response = client_bedrock_string.get('completion', 'Erro na resposta')

    # print(client_final_response2)

    return {
        'statusCode': 200,
        'body': (client_final_response2)
    }



### Seção 7 - Chatbot

In [ ]:
#Steps
#1 import the ConversationSummaryBufferMemory, ConversationChain, ChatBedrock (BedrockChat) Langchain Modules
from langchain.chains import ConversationChain
from langchain.memory import ConversationSummaryBufferMemory
from langchain_aws import ChatBedrock
#2a Write a function for invoking model- client connection with Bedrock with profile, model_id & Inference params- model_kwargs
def demo_chatbot():
    demo_llm=ChatBedrock(
       credentials_profile_name='default',
       model_id='anthropic.claude-3-haiku-20240307-v1:0',
       model_kwargs= {
           "max_tokens": 300,
           "temperature": 0.1,
           "top_p": 0.9,
           "stop_sequences": ["\n\nHuman:"]} )
    return demo_llm
#2b Test out the LLM with Predict method instead use invoke method
#     return demo_llm.invoke(input_text)
# response=demo_chatbot(input_text="Hi, what is the temperature in new york in January?")
# print(response)

# #3 Create a Function for  ConversationSummaryBufferMemory  (llm and max token limit)
def demo_memory():
    llm_data=demo_chatbot()
    memory=ConversationSummaryBufferMemory(llm=llm_data,max_token_limit=300)
    return memory

#4 Create a Function for Conversation Chain - Input text + Memory
def demo_conversation(input_text,memory):
    llm_chain_data=demo_chatbot()
    llm_conversation=ConversationChain(llm=llm_chain_data,memory=memory,verbose=True)

#5 Chat response using invoke (Prompt template)
    chat_reply=llm_conversation.invoke(input_text)
    return chat_reply['response']

# #1 https://python.langchain.com/v0.1/docs/integrations/llms/bedrock/
# #pip install -U langchain-aws
# #pip install anthropic


In [ ]:
# Source:Below code is provided by Streamlit and AWS 

#1 import streamlit and chatbot file
import streamlit as st 
import  chatbot_backend as demo  #**Import your Chatbot file as demo

#2 Set Title for Chatbot - https://docs.streamlit.io/library/api-reference/text/st.title
st.title("Hi, This is Chatbot Anisha :sunglasses:") # **Modify this based on the title you want in want

#3 LangChain memory to the session cache - Session State - https://docs.streamlit.io/library/api-reference/session-state
if 'memory' not in st.session_state: 
    st.session_state.memory = demo.demo_memory() #** Modify the import and memory function() attributes initialize the memory

#4 Add the UI chat history to the session cache - Session State - https://docs.streamlit.io/library/api-reference/session-state
if 'chat_history' not in st.session_state: #see if the chat history hasn't been created yet
    st.session_state.chat_history = [] #initialize the chat history

#5 Re-render the chat history (Streamlit re-runs this script, so need this to preserve previous chat messages)
for message in st.session_state.chat_history: 
    with st.chat_message(message["role"]): 
        st.markdown(message["text"]) 

#6 Enter the details for chatbot input box 
     
input_text = st.chat_input("Powered by Bedrock and Claude") # **display a chat input box
if input_text: 
    
    with st.chat_message("user"): 
        st.markdown(input_text) 
    
    st.session_state.chat_history.append({"role":"user", "text":input_text}) 

    chat_response = demo.demo_conversation(input_text=input_text, memory=st.session_state.memory) #** replace with ConversationChain Method name - call the model through the supporting library
    
    with st.chat_message("assistant"): 
        st.markdown(chat_response) 
    
    st.session_state.chat_history.append({"role":"assistant", "text":chat_response}) 

### Seção 8 - Vector Embeddings

In [ ]:
import json  # Biblioteca para manipulação de JSON
import boto3  # Biblioteca para interagir com serviços AWS

# Cria um cliente para o serviço Amazon Bedrock na região us-east-1
client = boto3.client('bedrock-runtime', region_name='us-east-1')

# ID do modelo da Amazon utilizado para gerar embeddings
model_id = "amazon.titan-embed-text-v2:0"

# Texto de entrada para ser processado pelo modelo
input_text = 'Hallo Welt!'

# Cria o dicionário de requisição no formato esperado pela API
native_request = {'inputText': input_text}

# Converte o dicionário para uma string JSON
request = json.dumps(native_request)

# Envia a requisição para o modelo da Amazon Bedrock
response = client.invoke_model(modelId=model_id, body=request)

# Lê a resposta da API e converte de JSON para um dicionário Python
model_response = json.loads(response['body'].read().decode('utf-8'))

# Exibe a resposta completa para depuração
print("Resposta completa do modelo:", model_response)

# Obtém os embeddings gerados pelo modelo (caso a chave exista, senão retorna uma lista vazia)
embeddings = model_response.get('embedding', [])

# Obtém a contagem de tokens de entrada (caso a chave exista, senão retorna um aviso)
input_token_count = model_response.get('inputTokenCount', 'Chave não encontrada')

# Exibe as informações processadas
print("\nSua entrada:")
print(input_text)
print(f"Número de tokens da entrada: {input_token_count}")
print(f"Tamanho da geração de embedding: {len(embeddings)}")
print("Embedding:")
print(embeddings)


### Seção 9 - RAG

Backend

In [ ]:
#1. Import OS, Document Loader, Text Splitter, Bedrock Embeddings, Vector DB, VectorStoreIndex, Bedrock-LLM
import os
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_aws import BedrockEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.indexes import VectorstoreIndexCreator
from langchain_aws import BedrockLLM
from langchain_aws import ChatBedrock 
 
#5c. Wrap within a function
def hr_index():
    #2. Define the data source and load data with PDFLoader(https://www.upl-ltd.com/images/people/downloads/Leave-Policy-India.pdf)
    data_load=PyPDFLoader('https://www.upl-ltd.com/images/people/downloads/Leave-Policy-India.pdf')  
 
    #3. Split the Text based on Character, Tokens etc. - Recursively split by character - ["\n\n", "\n", " ", ""]
    data_split=RecursiveCharacterTextSplitter(separators=["\n\n", "\n", " ", ""], chunk_size=100,chunk_overlap=10)
    #4. Create Embeddings -- Client connection
    data_embeddings=BedrockEmbeddings(
    credentials_profile_name= 'default',
    model_id='amazon.titan-embed-text-v2:0')
    #5à Create Vector DB, Store Embeddings and Index for Search - VectorstoreIndexCreator
    data_index=VectorstoreIndexCreator(
        text_splitter=data_split,
        embedding=data_embeddings,
        vectorstore_cls=FAISS)
    #5b  Create index for HR Policy Document
    db_index=data_index.from_loaders([data_load])
    return db_index
#6a. Write a function to connect to Bedrock Foundation Model - Claude Foundation Model
def hr_llm():
    llm=ChatBedrock(
        credentials_profile_name='default',
        model_id= 'anthropic.claude-3-haiku-20240307-v1:0',#'anthropic.claude-v2',
        model_kwargs={
        "max_tokens_to_sample":3000,
        "temperature": 0.1,
        "top_p": 0.9})
    return llm
#6b. Write a function which searches the user prompt, searches the best match from Vector DB and sends both to LLM.
def hr_rag_response(index,question):
    rag_llm=hr_llm()
    hr_rag_query=index.query(question=question,llm=rag_llm)
    return hr_rag_query
# Index creation --> https://api.python.langchain.com/en/latest/indexes/langchain.indexes.vectorstore.VectorstoreIndexCreator.htmlimport os


Frontend

In [ ]:
# The below frontend code is provided by AWS and Streamlit. I have only modified it to make it look attractive.
import streamlit as st 
import rag_backend as demo ### replace rag_backend with your backend filename

st.set_page_config(page_title="Pergunatas para o RH com RAG") ### Modify Heading

new_title = '<p style="font-family:sans-serif; color:Green; font-size: 42px;"> Pergunatas para o RH com RAG 🎯</p>'
st.markdown(new_title, unsafe_allow_html=True) ### Modify Title

if 'vector_index' not in st.session_state: 
    with st.spinner("📀 Espere pela magia...Todas as coisas bonitas da vida levam tempo :-)"): ###spinner message
        st.session_state.vector_index = demo.hr_index() ### Your Index Function name from Backend File

input_text = st.text_area("Input text", label_visibility="collapsed") 
go_button = st.button("📌Aprenda GenAI com Rahul Trisal", type="primary") ### Button Name

if go_button: 
    
    with st.spinner("📢Nascido para perder,viva para vencer. Lemmy Kilmister"): ### Spinner message
        response_content = demo.hr_rag_response(index=st.session_state.vector_index, question=input_text) ### replace with RAG Function from backend file
        st.write(response_content) 